<a href="https://colab.research.google.com/github/Tinotenda-29/TINOTENDA_MATIRAMBA_R229119Q_21_CNN_MODELS_ASSIGNMENT-1/blob/main/CNN_MODELS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%
# ==============================================================================
# STEP 1: SETUP, CONFIGURATION, AND IMPORTS (REVISED FOR LATE FUSION)
# ==============================================================================
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, applications, optimizers
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from PIL import Image
from tqdm import tqdm
from google.colab import drive
import matplotlib.pyplot as plt

# --- Code to Mount Google Drive (Run this first in Colab) ---
drive.mount('/content/gdrive')
# -----------------------------------------------------------

# --- Configuration ---
# CRITICAL: Ensure DRIVE_PATH points to the directory containing
# 'final_zimbabwe_property_listings_complete.csv' AND the 'images' folder.
DRIVE_PATH = '/content/gdrive/MyDrive/'
CSV_FILENAME = os.path.join(DRIVE_PATH, 'ZIMP.csv')
IMAGE_FOLDER_NAME = 'images'
# --- Hyperparameters (Tuning Applied) ---
IMG_WIDTH, IMG_HEIGHT, CHANNELS = 128, 128, 3
EPOCHS = 30 # Tuned value
BATCH_SIZE = 32
LEARNING_RATE = 5e-4 # Tuned value
FINE_TUNE = False

# List of CNN backbones to test
CNN_BACKBONES = [
    'AlexNet', 'NiN', 'ZNet', 'VGG', 'Google LeNet', 'InceptionV3',
    'Highway', 'InceptionV4', 'ResNet', 'InceptionV4 ResNet',
    'FractalNet', 'WideResNet', 'Xception', 'Residual Attention Neural Network',
    'Squeeze and Excitation Networks', 'Competitive Squeeze and Excitation Network',
    'DenseNet', 'MobileNet V2', 'Capsulenet', 'HRNet V2'
]

# Shared dense layer configurations
TABULAR_DENSE_LAYERS = [128, 64]
# PREDICTION_HEAD_LAYERS now acts as the prediction head layers for both branches
PREDICTION_HEAD_LAYERS = [128, 64]

print(f"Configuration Complete. Image size: {IMG_WIDTH}x{IMG_HEIGHT}. Epochs: {EPOCHS}, LR: {LEARNING_RATE}")
# Ensure we are in the correct directory for file operations
os.chdir(DRIVE_PATH)

# ==============================================================================
# STEP 2: DATA LOADING, PREPROCESSING, AND LOG SCALING
# ==============================================================================

print('\nLoading CSV data and preparing tabular features...')
df = pd.read_csv(CSV_FILENAME)

IMAGE_FILENAME_COLUMN = 'image_filenames'

# Drop rows where 'price' or the image filename is missing
df.dropna(subset=['price', IMAGE_FILENAME_COLUMN], inplace=True)

# Define features and target
tabular_features = [
    'bedrooms', 'bathrooms', 'land_area', 'building_area',
    'property_type', 'location'
]
target = 'price'

# Filter DataFrame and impute missing values
df_filtered = df[tabular_features + [target, IMAGE_FILENAME_COLUMN]].copy()

# Impute numerical features with the median
for col in ['bedrooms', 'bathrooms', 'land_area', 'building_area']:
    df_filtered.loc[:, col] = df_filtered[col].fillna(df_filtered[col].median())

# Impute categorical features with 'Unknown'
for col in ['property_type', 'location']:
    df_filtered.loc[:, col] = df_filtered[col].fillna('Unknown')

X = df_filtered[tabular_features]
y = df_filtered[target].values
image_filenames = df_filtered[IMAGE_FILENAME_COLUMN].values.astype(str)

# --- Define Preprocessing Pipelines for Tabular Features (X) ---
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# Apply preprocessing to X
X_tabular = preprocessor.fit_transform(X).toarray()
tabular_input_shape = (X_tabular.shape[1],)
print(f"Tabular data shape after preprocessing: {X_tabular.shape}")


# --- CRITICAL FIX: LOG TRANSFORM THE TARGET VARIABLE (y) ---
y_target_log = np.log1p(y)
target_scaler = StandardScaler()
y_target_scaled = target_scaler.fit_transform(y_target_log.reshape(-1, 1)).flatten()
y_target = y_target_scaled

print("\n✅ Target variable 'price' has been Log-Transformed and Standard Scaled for maximum stability.")


# ==============================================================================
# STEP 3: IMAGE AND MASK PREPARATION (REVISED FOR FULL DATASET)
# ==============================================================================

def load_and_preprocess_image_data_with_mask(filenames, drive_path=DRIVE_PATH, image_folder=IMAGE_FOLDER_NAME):
    """Loads images or a zero-image placeholder, and generates a binary mask."""
    image_dir = os.path.join(drive_path, image_folder)
    images = []
    image_masks = []

    available_files = set()
    try:
        if not os.path.exists(image_dir):
            raise FileNotFoundError(f"Image directory not found at {image_dir}")
        available_files = set(os.listdir(image_dir))
        print(f"Found image directory at {image_dir}.")
    except FileNotFoundError as e:
        print(f"ERROR: {e}. Assuming no images are available. Check DRIVE_PATH and image_folder.")

    print("\nPreparing image data with masking strategy...")

    for filename in tqdm(filenames, desc="Loading Images and Creating Masks"):
        path = filename

        if path in available_files:
            try:
                # Load, resize, and normalize the actual image
                img = Image.open(os.path.join(image_dir, path)).convert('RGB')
                img = img.resize((IMG_WIDTH, IMG_HEIGHT))
                img_arr = img_to_array(img).astype('float32') / 255.0
                images.append(img_arr)
                image_masks.append(1.0) # Mask is 1 (Image is present)
            except Exception:
                # Fallback if image exists but is corrupted
                images.append(np.zeros((IMG_HEIGHT, IMG_WIDTH, CHANNELS), dtype=np.float32))
                image_masks.append(0.0) # Treat as missing
        else:
            # Placeholder for missing images
            images.append(np.zeros((IMG_HEIGHT, IMG_WIDTH, CHANNELS), dtype=np.float32))
            image_masks.append(0.0) # Mask is 0 (Image is missing)

    image_data = np.array(images)
    image_masks = np.array(image_masks)
    image_input_shape = image_data.shape[1:]

    print(f"Total records processed: {len(image_data)}. Records with images: {int(np.sum(image_masks))}")
    print(f"Image data shape: {image_data.shape}. Mask shape: {image_masks.shape}")

    return image_data, image_input_shape, image_masks

# Execute loading and masking
image_data, image_input_shape, image_masks = load_and_preprocess_image_data_with_mask(image_filenames)

# Perform the final Train/Test Split (including the mask array)
X_tab_train, X_tab_test, X_img_train, X_img_test, y_train, y_test, X_mask_train, X_mask_test = train_test_split(
    X_tabular, image_data, y_target, image_masks, test_size=0.2, random_state=42
)
print(f"\nTraining samples: {len(y_train)}, Test samples: {len(y_test)}")

# ==============================================================================
# STEP 4: CNN STUB AND MODEL BUILDER DEFINITIONS (REVISION: HUBER Loss + LATE FUSION)
# ==============================================================================

# --- CNN Stubs (Feature Extraction) ---

def nin_block(x, num_channels, block_name):
    x = layers.Conv2D(num_channels, (5, 5), activation='relu', padding='same', name=f'{block_name}_conv1')(x)
    x = layers.Conv2D(num_channels, (1, 1), activation='relu', padding='same', name=f'{block_name}_conv2')(x)
    x = layers.Conv2D(num_channels, (1, 1), activation='relu', padding='same', name=f'{block_name}_conv3')(x)
    return x

def nin_net(input_tensor, name='NiN'):
    x = nin_block(input_tensor, 32, block_name=f'{name}_b1')
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = nin_block(x, 64, block_name=f'{name}_b2')
    x = layers.GlobalAveragePooling2D(name=f'{name}_avg_pool')(x)
    return x

def alexnet_block(input_tensor, name='AlexNet'):
    x = layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', name=f'{name}_conv1')(input_tensor)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), name=f'{name}_pool1')(x)
    x = layers.Conv2D(256, (5, 5), activation='relu', padding='same', name=f'{name}_conv2')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), name=f'{name}_pool2')(x)
    x = layers.Conv2D(384, (3, 3), activation='relu', padding='same', name=f'{name}_conv3')(x)
    x = layers.Conv2D(384, (3, 3), activation='relu', padding='same', name=f'{name}_conv4')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name=f'{name}_conv5')(x)
    x = layers.GlobalAveragePooling2D(name=f'{name}_avg_pool')(x)
    return x

# Placeholder stubs for other models
def znet_stub(input_tensor):
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    return x
def googlenet_stub(input_tensor):
    x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_tensor)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    return x
def highway_stub(input_tensor):
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
    x = layers.GlobalAveragePooling2D()(x)
    return x
def inceptionv4_stub(input_tensor):
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
    x = layers.GlobalAveragePooling2D()(x)
    return x
def fractalnet_stub(input_tensor):
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
    x = layers.GlobalAveragePooling2D()(x)
    return x
def wideresnet_stub(input_tensor):
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
    x = layers.GlobalAveragePooling2D()(x)
    return x
def res_attention_stub(input_tensor):
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
    x = layers.GlobalAveragePooling2D()(x)
    return x
def senet_stub(input_tensor):
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
    x = layers.GlobalAveragePooling2D()(x)
    return x
def comp_senet_stub(input_tensor):
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
    x = layers.GlobalAveragePooling2D()(x)
    return x
def capsulenet_stub(input_tensor):
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
    x = layers.GlobalAveragePooling2D()(x)
    return x
def hrnet_stub(input_tensor):
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
    x = layers.GlobalAveragePooling2D()(x)
    return x


# --- Dual Input - Model Builder (LATE FUSION) ---
def build_fusion_model_with_cnn(cnn_name, tab_shape, img_shape, fine_tune_flag, tab_units, prediction_units):
    """Creates a Dual Input - Late Prediction Fusion model with masking and Huber loss."""

    safe_cnn_name = cnn_name.replace(' ', '_').replace('-', '_')

    # --- Branch 1: Tabular Feature Extraction (MLP) ---
    tab_input = layers.Input(shape=tab_shape, name="tabular_input")
    x_tab = tab_input
    for units in tab_units:
        x_tab = layers.Dense(units, activation='relu')(x_tab)
        x_tab = layers.Dropout(0.3)(x_tab)
    # Final feature vector for tabular branch
    tab_features = layers.Dense(64, activation='relu', name="tab_dense_out")(x_tab)

    # --- Branch 2: Image Feature Extraction (CNN) ---
    img_input = layers.Input(shape=img_shape, name="image_input")

    # --- Branch 3: Mask Input (Scalar 0 or 1) ---
    mask_input = layers.Input(shape=(1,), name="mask_input")

    # A. Keras Applications Mapping (Pre-trained models)
    BACKBONE_MAP = {
        'VGG': applications.VGG16,
        'InceptionV3': applications.InceptionV3,
        'ResNet': applications.ResNet50,
        'InceptionV4 ResNet': applications.InceptionResNetV2,
        'Xception': applications.Xception,
        'DenseNet': applications.DenseNet121,
        'MobileNet V2': applications.MobileNetV2
    }

    x_img = None
    if cnn_name in BACKBONE_MAP:
        BaseModel = BACKBONE_MAP[cnn_name]
        base_cnn = BaseModel(
            weights='imagenet', include_top=False, input_tensor=img_input
        )
        base_cnn.trainable = fine_tune_flag
        x_img = base_cnn.output
        x_img = layers.GlobalAveragePooling2D(name=f'{safe_cnn_name}_avg_pool')(x_img)

    # B. Custom Stubs Mapping
    elif cnn_name == 'AlexNet':
        x_img = alexnet_block(img_input)
    elif cnn_name == 'NiN':
        x_img = nin_net(img_input)
    elif cnn_name == 'ZNet':
        x_img = znet_stub(img_input)
    elif cnn_name == 'Google LeNet':
        x_img = googlenet_stub(img_input)
    elif cnn_name == 'Highway':
        x_img = highway_stub(img_input)
    elif cnn_name == 'InceptionV4':
        x_img = inceptionv4_stub(img_input)
    elif cnn_name == 'FractalNet':
        x_img = fractalnet_stub(img_input)
    elif cnn_name == 'WideResNet':
        x_img = wideresnet_stub(img_input)
    elif cnn_name == 'Residual Attention Neural Network':
        x_img = res_attention_stub(img_input)
    elif cnn_name == 'Squeeze and Excitation Networks':
        x_img = senet_stub(img_input)
    elif cnn_name == 'Competitive Squeeze and Excitation Network':
        x_img = comp_senet_stub(img_input)
    elif cnn_name == 'Capsulenet':
        x_img = capsulenet_stub(img_input)
    elif cnn_name == 'HRNet V2':
        x_img = hrnet_stub(img_input)
    else:
        raise ValueError(f"Unknown CNN backbone: {cnn_name}")

    # Feature vector for image branch
    img_features = layers.Dense(64, activation='relu', name="img_dense_pre_mask")(x_img)

    # CRITICAL MASKING STEP: Disable image features if mask is 0
    img_features_masked = layers.multiply([img_features, mask_input], name="masked_img_features")

    # 🔥 LATE PREDICTION FUSION ARCHITECTURE 🔥

    # 1. Tabular Prediction Head
    x_tab_pred = tab_features
    for units in prediction_units:
        x_tab_pred = layers.Dense(units, activation='relu')(x_tab_pred)
    tab_output = layers.Dense(1, name="tabular_prediction_output")(x_tab_pred)

    # 2. Image Prediction Head
    x_img_pred = img_features_masked
    for units in prediction_units:
        x_img_pred = layers.Dense(units, activation='relu')(x_img_pred)
    img_output = layers.Dense(1, name="image_prediction_output")(x_img_pred)

    # 3. Hybridization/Averaging (combines the two individual predictions)
    # The final prediction is the average of the two separate learning pathways.
    final_output = layers.Average(name="final_hybrid_prediction")([tab_output, img_output])

    # Build and compile the model
    model = models.Model(inputs=[tab_input, img_input, mask_input], outputs=final_output, name=safe_cnn_name)

    optimizer = optimizers.Adam(learning_rate=LEARNING_RATE)

    # Use Huber Loss for robustness
    model.compile(loss=tf.keras.losses.Huber(delta=1.0), optimizer=optimizer, metrics=['mae'])
    return model


# ==============================================================================
# STEP 5: TRAINING, PREDICTION, AND EVALUATION LOOP
# ==============================================================================
results = []
best_model_name = None
best_rmse = np.inf
best_history = None
best_predictions = None
best_true_values = None

print(f"\nStarting sequential training of the 20 CNN-backed models (Tuning: EPOCHS={EPOCHS}, LR={LEARNING_RATE}, Loss=Huber, Architecture=Late Fusion)...")

for i, cnn_name in enumerate(CNN_BACKBONES):
    # Clear session to prevent memory leaks/corruption between models
    K.clear_session()
    print(f"\n================ Model {i+1}/20: {cnn_name} ================")

    try:
        model = build_fusion_model_with_cnn(
            cnn_name=cnn_name,
            tab_shape=tabular_input_shape,
            img_shape=image_input_shape,
            fine_tune_flag=FINE_TUNE,
            tab_units=TABULAR_DENSE_LAYERS,
            prediction_units=PREDICTION_HEAD_LAYERS # Using prediction_units for the fusion head
        )

        # Define the training and validation data dictionaries (including mask)
        train_data = {
            "tabular_input": X_tab_train,
            "image_input": X_img_train,
            "mask_input": X_mask_train.reshape(-1, 1) # Mask must be reshaped to (N, 1)
        }
        test_data = {
            "tabular_input": X_tab_test,
            "image_input": X_img_test,
            "mask_input": X_mask_test.reshape(-1, 1)  # Mask must be reshaped to (N, 1)
        }

        # 2. Train the Model
        print(f"Training {cnn_name}...")
        history = model.fit(
            x=train_data,
            y=y_train,
            validation_data=(test_data, y_test),
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            verbose=2
        )

        # 3. Predict on Test Set (Scaled Output)
        y_pred_scaled = model.predict(test_data).flatten()

        # 4. Inverse Transform Predictions and True Values (Log Scaling Inverse)
        y_test_log = target_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
        y_pred_log = target_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

        # Step 4b: Inverse Log the values to get original currency (exp(x) - 1)
        y_test_original = np.expm1(y_test_log)
        y_pred_original = np.expm1(y_pred_log)

        # Ensure prices are non-negative
        y_test_original[y_test_original < 0] = 0
        y_pred_original[y_pred_original < 0] = 0

        # 5. Evaluate with Original Values
        mae = mean_absolute_error(y_test_original, y_pred_original)
        mse = mean_squared_error(y_test_original, y_pred_original)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test_original, y_pred_original)

        print(f"Training Complete. | *RMSE: {rmse:,.2f}* | MAE: {mae:,.2f} | R2: {r2:.4f}")

        # Store results and track the best model
        results.append({
            'Model Name': cnn_name,
            'RMSE': rmse, 'MAE': mae, 'R2': r2, 'Notes': 'Successful'
        })
        if rmse < best_rmse:
            best_rmse = rmse
            best_model_name = cnn_name
            best_history = history
            best_predictions = y_pred_original
            best_true_values = y_test_original

    except Exception as e:
        print(f"❌ Error training {cnn_name}. Skipping. Error: {e}")
        results.append({
            'Model Name': cnn_name, 'RMSE': np.nan, 'MAE': np.nan, 'R2': np.nan,
            'Notes': f'Failed: {e}'
        })


# ==============================================================================
# STEP 6: REPORTING AND VISUALIZATION
# ==============================================================================

if results:
    results_df = pd.DataFrame(results).sort_values(by='RMSE', ascending=True)

    print("\n" + "="*50)
    print("             MODEL PERFORMANCE SUMMARY")
    print("="*50)
    print(results_df.to_markdown(index=False))
    print("="*50)

    # Check if a best model was successfully trained
    if best_model_name and not np.isnan(best_rmse):
        print(f"🥇 Best Model: {best_model_name} (RMSE: {best_rmse:,.2f})")
    else:
        print("🚨 Warning: No models completed successfully or a new best model could not be determined.")

    print("="*50)

    # 1. Plotting the performance of the best model (Actual vs. Predicted)
    if best_predictions is not None:
        plt.figure(figsize=(10, 6))
        plt.scatter(best_true_values, best_predictions, alpha=0.5)

        min_val = min(best_true_values.min(), best_predictions.min())
        max_val = max(best_true_values.max(), best_predictions.max())

        plt.plot([min_val, max_val], [min_val, max_val], 'r--')
        plt.xlabel("Actual Price (USD)")
        plt.ylabel("Predicted Price (USD)")
        plt.title(f"Actual vs. Predicted Prices for Best Model: {best_model_name}")
        plt.grid(True)
        plt.savefig('best_model_actual_vs_predicted.png')
        plt.close()
        print("\n✅ Saved: best_model_actual_vs_predicted.png")

    # 2. Plotting all model RMSEs
    plt.figure(figsize=(12, 6))
    valid_results = results_df.dropna(subset=['RMSE'])
    plt.bar(valid_results['Model Name'], valid_results['RMSE'], color='skyblue')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("Root Mean Squared Error (RMSE)")
    plt.title("Comparison of Model Performance (RMSE)")
    plt.tight_layout()
    plt.savefig('all_model_rmse_comparison.png')
    plt.close()
    print("✅ Saved: all_model_rmse_comparison.png")

    # 3. Plotting Training History (Loss) of the Best Model
    if best_history is not None:
        plt.figure(figsize=(8, 5))
        # Note: The loss here is the scaled Huber loss, but still shows learning trend
        plt.plot(best_history.history['loss'], label='Train Loss (Scaled Huber)')
        plt.plot(best_history.history['val_loss'], label='Validation Loss (Scaled Huber)')
        plt.title(f'{best_model_name} Training History (Log-Scaled Huber Loss)')
        plt.xlabel('Epoch')
        plt.ylabel('Loss (Huber)')
        plt.legend()
        plt.grid(True)
        plt.savefig('best_model_training_history.png')
        plt.close()
        print("✅ Saved: best_model_training_history.png")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Configuration Complete. Image size: 128x128. Epochs: 8, LR: 0.01

Loading CSV data and preparing tabular features...
Tabular data shape after preprocessing: (1591, 104)

✅ Target variable 'price' has been Log-Transformed and Standard Scaled for maximum stability.
Found image directory at /content/gdrive/MyDrive/images.

Preparing image data with masking strategy...


Loading Images and Creating Masks: 100%|██████████| 1591/1591 [00:15<00:00, 101.58it/s] 


Total records processed: 1591. Records with images: 466
Image data shape: (1591, 128, 128, 3). Mask shape: (1591,)

Training samples: 1272, Test samples: 319

Starting sequential training of the 20 CNN-backed models (Tuning: EPOCHS=8, LR=0.01, Loss=Huber, Architecture=Late Fusion)...

================ Model 1/20: AlexNet ================
Training AlexNet...
Epoch 1/8
40/40 - 59s - 1s/step - loss: 7.0722 - mae: 7.4195 - val_loss: 0.2924 - val_mae: 0.6106
Epoch 2/8
40/40 - 57s - 1s/step - loss: 0.3262 - mae: 0.6631 - val_loss: 0.2956 - val_mae: 0.6054
Epoch 3/8
40/40 - 81s - 2s/step - loss: 0.3129 - mae: 0.6381 - val_loss: 0.2996 - val_mae: 0.6337
Epoch 4/8
40/40 - 81s - 2s/step - loss: 0.3123 - mae: 0.6330 - val_loss: 0.2873 - val_mae: 0.6050
Epoch 5/8
40/40 - 82s - 2s/step - loss: 0.3088 - mae: 0.6305 - val_loss: 0.2856 - val_mae: 0.5934
Epoch 6/8
40/40 - 58s - 1s/step - loss: 0.3016 - mae: 0.6185 - val_loss: 0.2861 - val_mae: 0.5924
Epoch 7/8
40/40 - 80s - 2s/step - loss: 0.3045 - mae

/tmp/ipython-input-596566861.py:297: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_cnn = BaseModel(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training MobileNet V2...
Epoch 1/8
40/40 - 30s - 754ms/step - loss: 0.3766 - mae: 0.7282 - val_loss: 0.2974 - val_mae: 0.6359
Epoch 2/8
40/40 - 39s - 968ms/step - loss: 0.3131 - mae: 0.6368 - val_loss: 0.3030 - val_mae: 0.6250
Epoch 3/8
40/40 - 20s - 508ms/step - loss: 0.2989 - mae: 0.6193 - val_loss: 0.2849 - val_mae: 0.6022
Epoch 4/8
40/40 - 19s - 486ms/step - loss: 0.2873 - mae: 0.5991 - val_loss: 0.3076 - val_mae: 0.6351
Epoch 5/8
40/40 - 23s - 574ms/step - loss: 0.2664 - mae: 0.5705 - val_loss: 0.2898 - val_mae: 0.6023
Epoch 6/8
40/40 - 38s - 959ms/step - loss: 0.2690 - mae: 0.5724 - val_loss: 0.2995 - val_mae: 0.6145
Epoch 7/8
40/40 - 21s - 523ms/step - loss: 0.2570 - mae: 0.5543 - val_loss: 0.2994 - val_mae: 0.6191
Epoch 8/8
40/40 - 21s - 537ms/step - loss: 0.2570 - mae: 0.5536 - val_loss: 0.3165 - val_mae: 0.6325
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 489ms/step
Training Complete. | *RMSE: 4,462,011.76* | MAE: 684,904.02 | R2: -0.0181

==

Summary of Model Performance

The analysis summarizes the performance of 20 Hybrid Dual-Input Regression Models designed to predict property prices, ranked by the Root Mean Squared Error (RMSE).Key FindingsBest Performer: AlexNet achieved the lowest RMSE of 4,428,327.46.Ranking: The models clustered closely together, with the best (AlexNet) and the worst (InceptionV3, RMSE: 4,528,834.40) models showing little difference in error magnitude.Critical Implication: Negative $\mathbf{R}^2$A major finding is that all 20 models exhibited negative $\mathbf{R}^2$ scores, ranging from -0.0027 to -0.0488.A negative $\text{R}^2$ means that every model performs worse than a simple baseline that just predicts the average price of all properties.This indicates that the models, despite using complex CNN architectures (like VGG, ResNet, etc.) alongside tabular data, failed to capture the underlying variance in the property prices.


Conclusion


While AlexNet is statistically the best-performing option in this test set, none of the models are suitable for deployment. The extremely high RMSE (in the millions) and the universally negative $\text{R}^2$ scores suggest fundamental issues, likely related to:

Limited Data Sample: The necessary cleaning steps, including dropping rows with missing prices or images, resulted in a dataset that was too small for effectively training complex, high-capacity hybrid models (like those using deep CNNs).

Image Feature Contamination: The models incorrectly extracted features because many property images contained more than one house or had irrelevant background details. This meant the CNNs picked up features from the wrong structure, corrupting the visual signal and making the late fusion process unreliable